## 随机造一些数据用于DRG分组

### 0. 预期目标数据形式 Target to generate(simplified)
> infants not included<br>
> fee (out of pocket,insurance,etc all in one) <br>
> intubation/ICU cases not included
> only one operation

|caseid|gender|age|diag_p(rimary)|diag_oth(er)|oper_p(rimary)|oper_oth(er)|fee|hosp_id|mdci|adrgi|
|----|----|----|----|----|----|----|----|----|----|----|
|data|data|data|data|data|data|null|data|data|data|data|

In [1]:
import numpy as np
import pandas as pd

### 1. 生成预期费用
> 按MDC -> ADRG -> DRG -> Case 层级继承并变异思路虚构

#### 1.1. 随机生成MDC费用

In [3]:
mdc_lst=pd.read_csv('./LOOKUP_TABLES/1_MDC_LIST_GB2312.csv',encoding='gbk')

In [4]:
mdc_lst.head()

,NO,MDC_CODE,MDC_NAME
0,1,MDCA,先期分组疾病及相关操作
1,2,MDCB,神经系统疾病及功能障碍
2,3,MDCC,眼疾病及功能障碍
3,4,MDCD,头颈、耳、鼻、口、咽疾病及功能障碍
4,5,MDCE,呼吸系统疾病及功能障碍


In [5]:
# gen mean fee of each mdc
mdc_means=np.random.randint(100, high=1000, size=len(mdc_lst), dtype=int)
mdc_devs=np.random.randint(1, high=20, size=len(mdc_lst), dtype=int)
mdc_mean_dict={mdc:(m,dev) for mdc,m,dev in zip(mdc_lst.MDC_CODE,mdc_means,mdc_devs)}

In [11]:
mdc_mean_dict['MDCA']

(360, 15)

In [12]:
#gen rand caseid
def ranstr(num):
    H = list('ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz0123456789')
    salt = ''
    for i in range(num):
        salt += np.random.choice(H)
    return salt

> 修正了一些原lookup table的数据的错误
```python
icd_mdc_adrg=pd.read_csv(./LOOKUP_TABLES/2_MDC_to_ADRG_GB2312.csv',encoding='gbk')
icd_mdc=icd_mdc_adrg.loc[icd_mdc_adrg.is_icd_mdc==False].copy()
icd_mdc_icd9_adrg=icd_mdc_adrg.loc[icd_mdc_adrg.is_icd_mdc==True].copy()

icd_mdc.to_csv('./LOOKUP_TABLES/1_5_ICD_to_MDC.csv',encoding='gbk')
icd_mdc_icd9_adrg.to_csv('./LOOKUP_TABLES/2_5_MDC_ICD9_to_ADRG.csv',encoding='gbk')
```

#### 1.2. 依据MDC费用分布随机生成ADRG费用

In [21]:
mdc2adrg=pd.read_csv('./LOOKUP_TABLES/2_MDC_to_ADRG_GB2312.csv',encoding='utf-8')

In [22]:
mdc2adrg.head()

,MDC_CODE,MDC_NAME,ADRG_CODE,ADRG_NAME
0,MDCA,先期分组疾病及相关操作,AA1,心脏移植
1,MDCA,先期分组疾病及相关操作,AB1,肝移植
2,MDCA,先期分组疾病及相关操作,AC1,胰/肾联合移植
3,MDCA,先期分组疾病及相关操作,AD1,胰腺移植
4,MDCA,先期分组疾病及相关操作,AE1,肾移植


In [28]:
adrg_means=[np.random.normal(*(mdc_mean_dict.get('MDC'+adrg[0]))) for adrg in mdc2adrg.ADRG_CODE]
adrg_devs=np.random.randint(1, high=5, size=len(mdc2adrg), dtype=int)
adrg_mean_dict={adrg:(m,dev) for adrg,m,dev in zip(mdc2adrg.ADRG_CODE,adrg_means,adrg_devs)}

In [91]:
adrg_mean_dict['AA1']

(338.1095244907688, 4)

#### 1.3. 依据ADRG费用分布随机生成DRG费用

In [100]:
adrg2drg= pd.read_csv('./LOOKUP_TABLES/4_ADRG_to_DRG_GB2312.csv',encoding='gbk')

In [101]:
adrg2drg.head()

,ADRG_CODE,DRG_CODE,DRG_NAME
0,AA1,AA19,心脏移植
1,AB1,AB19,肝移植
2,AC1,AC19,胰/肾联合移植
3,AD1,AD19,胰腺移植
4,AE1,AE19,肾移植


In [102]:
drg_means=[np.random.normal(*(adrg_mean_dict.get(adrg))) for adrg in adrg2drg.ADRG_CODE]
drg_devs=np.random.randint(1, high=5, size=len(adrg2drg), dtype=int)
drg_mean_dict={drg:(m,dev) for drg,m,dev in zip(adrg2drg.DRG_CODE,drg_means,drg_devs)}

In [103]:
drg_mean_dict['AA19']

(338.12894280172344, 2)

### 2. 制作映射表（采样函数）
> 按MDC -> ADRG -> DRG -> Case 层级采样

#### 2.1. MDC to ADRG

In [44]:
mdc_adrg_dict=mdc2adrg.groupby('MDC_CODE')['ADRG_CODE'].apply(list)
mdc_adrg_dict=mdc_adrg_dict.to_dict()

> Test

In [59]:
mdc_adrg_dict.get('MDCA')

['AA1', 'AB1', 'AC1', 'AD1', 'AE1', 'AF1', 'AG1', 'AG2', 'AH1']

#### 2.2. ADRG to DRG

In [104]:
# adrg to drg mapping
adrg_drg_dict=adrg2drg.groupby('ADRG_CODE')['DRG_CODE'].apply(list)
adrg_drg_dict=adrg_drg_dict.to_dict()

> Test

In [105]:
adrg_drg_dict['AA1']

['AA19']

#### 2.3. ADRG to ICD

In [62]:
## load adrg icd icd9 lookup table
icd2mdc2adrg=pd.read_csv('./LOOKUP_TABLES/2_5_MDC_ICD9_to_ADRG.csv',encoding='utf-8')

In [63]:
icd2mdc2adrg.head()

,MDC_CODE,MDC_NAME,ADRG_CODE,ADRG_NAME,ICD_CODE,ICD_DESC,AND_FLAG,GROUP_FLAG,is_icd_mdc,is_icd9
0,MDCA,先期分组疾病及相关操作,AA1,心脏移植,37.51,心脏移植术,N,0,True,True
1,MDCA,先期分组疾病及相关操作,AB1,肝移植,50.51,辅助肝移植,N,0,True,True
2,MDCA,先期分组疾病及相关操作,AB1,肝移植,50.5100x001,同种异体原位肝移植术,N,0,True,True
3,MDCA,先期分组疾病及相关操作,AB1,肝移植,50.5900x001,肝肾联合移植术,N,0,True,True
4,MDCA,先期分组疾病及相关操作,AB1,肝移植,50.5900x004,同种异体肝肾联合移植术,N,0,True,True


In [65]:
icd2mdc2adrg_icd=icd2mdc2adrg.loc[icd2mdc2adrg.is_icd9==False]
icd2mdc2adrg_icd9=icd2mdc2adrg.loc[icd2mdc2adrg.is_icd9==True]

In [66]:
# adrg to icd mapping
adrg_icd_dict=icd2mdc2adrg_icd.groupby('ADRG_CODE')['ICD_CODE'].apply(list)
adrg_icd_dict=adrg_icd_dict.to_dict()

In [67]:
# adrg to icd9 mapping
adrg_icd9_dict=icd2mdc2adrg_icd9.groupby('ADRG_CODE')['ICD_CODE'].apply(list)
adrg_icd9_dict=adrg_icd9_dict.to_dict()

> Test

In [123]:
adrg_icd_dict.get('BB1',['NA'])[:5]

['S01.800x011', 'S01.800x021', 'S01.800x031', 'S01.800x081', 'S01.801']

In [125]:
adrg_icd9_dict.get('BB2')[:5]

['1.15', '01.1800x002', '01.2100x001', '1.23', '01.2400x005']

#### 2.4. MCC/CC list

In [85]:
# load mcc cc lst

In [84]:
mcc_lst=pd.read_csv('./LOOKUP_TABLES/5_MCC_LIST_GB2312.csv',encoding='gbk')
cc_lst=pd.read_csv('./LOOKUP_TABLES/6_CC_LIST_GB2312.csv',encoding='gbk')

In [87]:
mcc_lst.head()

,MCC_ICD_CODE,MCC_ICD_NAME,EXCLUDE
0,A02.100,沙门菌脓毒症,7_1
1,A02.100x002,鼠伤寒沙门菌脓毒症,7_1
2,A02.101,猪霍乱沙门菌脓毒症,7_1
3,A03.800x001,不定型志贺菌痢疾,7_1
4,A03.800x002,菌痢混合感染,7_1


In [88]:
cc_lst.head()

,CC_ICD_CODE,CC_ICD_NAME,CC_EXCLUDE
0,A03.000x001,痢疾志贺菌痢疾,7_1
1,A03.200x001,鲍氏志贺菌痢疾,7_1
2,A03.900,细菌性痢疾,7_1
3,A03.900x002,慢性细菌性痢疾急性发作,7_1
4,A03.900x005,慢性隐匿型菌痢,7_1


### 3. 生成虚拟数据

In [ ]:
# mdc -> adrg -> fake drg(>50)fee -> case fee
# just use drg code as bridge no essential meaning

In [36]:
sample_size=10000

In [36]:
case_ids=[ranstr(5) for i in range(sample_size)]

case_mdcs=[np.random.choice(mdc_lst.MDC_CODE) for i in range(sample_size)]

case_adrgs=[np.random.choice(mdc_adrg_dict[m]) for m in case_mdcs]

In [ ]:
# diag 先只能不严谨的造了，后续改进

In [74]:
case_diag_p=[np.random.choice(adrg_icd_dict.get(adrg,['NA'])) for adrg in case_adrgs]

In [75]:
case_oper_p=[np.random.choice(adrg_icd9_dict.get(adrg,['NA'])) for adrg in case_adrgs]

In [ ]:
# 其他诊断也先不严谨先用cc/mcc造了，后续有效性靠分组器本身

In [89]:
case_diag_oth=[np.random.choice(mcc_lst.MCC_ICD_CODE) for i in range(sample_size)]

In [94]:
case_age=[np.random.randint(18,80) for i in range(sample_size)]

In [96]:
# 0 for female, 1 for male
case_gender=[np.random.choice([0,1]) for i in range(sample_size)]

In [112]:
# fake drg just a mark
case_drg=[adrg_drg_dict.get(adrg)[0] if age<=17 \
          else adrg_drg_dict.get(adrg)[-1] if age<=60 \
          else np.random.choice(adrg_drg_dict.get(adrg)) for adrg,age in zip(case_adrgs,case_age)]

In [114]:
# fee gen
case_fee=[np.random.normal(*(drg_mean_dict.get(drg))) for drg in case_drg]

In [118]:
## i means imaginary ii means imaginary^2
case_df=pd.DataFrame({"case_id":case_ids,"gender":case_gender,"age":case_age,\
                    "diag_p":case_diag_p,"diag_oth":case_diag_oth,\
                    "oper_p":case_oper_p,"fee":case_fee,\
                    "mdci":case_mdcs,"adrgi":case_adrgs,"drgii":case_drg})

In [119]:
case_df.head()

,case_id,gender,age,diag_p,diag_oth,oper_p,fee,mdci,adrgi,drgii
0,M2kT0,1,28,NA,D68.205,NA,270.346024,MDCZ,ZZ1,ZZ15
1,hnvKA,0,50,NA,S25.301,NA,134.990506,MDCS,SB1,SB15
2,UccRl,1,61,NA,S12.700,14.3901,157.395593,MDCC,CB1,CB19
3,lWS3r,0,23,NA,S35.201,54.1904,267.941256,MDCM,MJ1,MJ19
4,ectQx,0,78,D37.019,T82.501,NA,389.045262,MDCD,DR1,DR13


In [121]:
case_df.to_csv('./dummy_data/dummy_cases.csv',index=False)

In [ ]:
# 生成的缺失主诊断的数据姑且当作脏数据由野生分组器处理看看

### End